In [44]:
import numpy as np
import pandas as pd
import json
import os
#import pdb

pd.options.mode.chained_assignment = None

In [45]:
with open('config.json') as f:
    config = json.load(f)

in_file = config['pdf_input_location']
password = config['password']
file_list = [x.split('.')[0] for x in os.listdir(in_file['pdf_input_location'])]
out_file = config['csv_output_location']

In [46]:
#Concatenated yearly statment
df_in_yrly_stmt = pd.read_csv(out_file['csv_output_location']+'consolidated_yrly_stmt_outfrom_pdf.csv')

#Deposits\dr\debits in account
df_in_yrly_stmt_dr = df_in_yrly_stmt[df_in_yrly_stmt['Deposit Amount'].str.replace(',', '').astype(float) > 0.00]
df_in_yrly_stmt_dr['Narration'] = df_in_yrly_stmt_dr['Narration'].str.replace('\n','')

#Deposit lookup which has deposit label defined by user
df_in_depost_lookup = pd.read_csv(in_file['pdf_input_location']+'deposit labels lookup.csv')
df_in_depost_lookup = df_in_depost_lookup.apply(lambda x: x.astype(str).str.upper())
lkp_dct = df_in_depost_lookup.to_dict()

In [47]:
for i in lkp_dct:
    for j in lkp_dct[i]:
        df_in_yrly_stmt_dr.loc[df_in_yrly_stmt_dr['Narration'].str.contains(lkp_dct['Input text'][j]),'Deposit disciription'] = lkp_dct['New text'][j]
    break
df_in_yrly_stmt_dr = df_in_yrly_stmt_dr.sort_values(by=['Deposit disciription'])

In [48]:
df_in_yrly_stmt_dr.to_csv(out_file['csv_output_location']+'02 debit_operations_on_concatenated_stmt.csv',header=True,index=False)

In [49]:
%reset -f